**ЗАГРУЖАЕМ ДАТАСЕТ**
---

In [1]:
import json

def load_data(path):
    with open(path, encoding='utf-8') as file:
        data = json.load(file)
    return data

In [2]:
path = 'cintra_phoenix_oils_hr_mgck_feather.json'
dataset = load_data(path)

print(f'first element: \n{dataset[0]}')
print(f'Types: \n{type(dataset).__name__} of {type(dataset[0]).__name__} elements')

first element: 
{'id': 0, 'quote': '«У ннас среди ночi в райооне 55 часов упала полxа с водой в сттеклянной  таре, тоесть, посреди этой лужа, сстеккла, всёё, это уже поод утро, кстати, ээто не перрвыи раз,,, они сами по себе падают, как-то неправилььно рассчитывают, мы же должны выставлять по определённой картинке, у нас при мне было уже пару раз, чтоо сами по себе грохаются эти полки с буттылками» \n\n\n'}
Types: 
list of dict elements


**ПРЕДОБРАБОТКА ДАННЫХ**
---

---

**На этапе предобработки возникает несколько сложностей и спорных моментов. Начну с этапов, которые не требуют особых пояснений:**

1. Разбиваю предложения на токены (функция `tokenize`).

2. Привожу токены к нижнему регистру (функция `to_lower`).

3. Теперь немного контекста: в тексте встречается множество слов, в которых русские буквы заменены на их "английские аналоги", поэтому эмпирическим путем был создан словарь соответствий англ→рус (функция `en_ru_mapping`).

4. Похожая проблема связана с заменой цифр на буквы, здесь есть некая закономерность, которую можно описать так (функция `replace_numbers`):
    - '3' → 'З', если перед '3' стоит гласная (с '3' есть одно исключение, но оно не критично);
    - '3' → 'Е', если перед '3' стоит согласная;
    - '0' → 'О';
    - '1' → 'И';
    - '7' → 'Т'.
    - Есть также замены с '4', но их очень мало, поэтому можно игнорировать.

5. После замены всех потенциально информативных цифр и англоязычных букв на русские удаляю все символы, которые не являются русскими буквами.

6. Удаляю повторяющиеся буквы (например, "приииивет"), из-за этого могут пострадать слова с удвоенными буквами, однако эта потеря влияет на качество датасета гораздо меньше, чем наличие произвольного количества повторений (функция `remove_repeating_letters`).

7. Исправляю грамматические ошибки с помощью `Yandex.Speller`. Отмечу, что в тексте есть много ошибок, которые не удается исправить (подробнее об этом ниже). После `spell_correction` снова использую `tokenize`, так как внутри функции токены объединяются в строку для ускорения работы (ускорение в ~ x30).

8. Стемминг (приведение слова к базовой форме), как по мне, здесь малоэффективен, поэтому было решено от него отказаться (подробнее о причинах ниже) (функция `stemming`).

9. Лемматизация стандартизирует слова, приводя их к единой форме; считаю, что это эффективнее, чем стемминг. Пробовал разные комбинации: только стемминг, стемминг + лемматизация и только лемматизацию. Последний вариант оказался лучше (оценивал по получившимся тематикам) (функция `lemmatize`).

10. Удаляю стоп-слова (функция `remove_stop_words`).

---

**ПРОБЛЕМЫ**

В тексте множество грамматических ошибок, из-за чего эффективность стемминга и лемматизации снижается (не обрабатываются слова с ошибками → не стандартизируются нужные слова → не выполняется их основная функция), также не удаляются стоп-слова с ошибками, что добавляет тексту лишний шум.

Для решения этой проблемы искал `Spell Corrector`, удовлетворяющий следующим требованиям:
1. Поддержка русского языка;
2. Высокая скорость работы и небольшой размер.

В ходе поиска протестировал следующие инструменты:
- `python-language-tools`: запускает локальный сервер на Java, запросы к нему работают медленно, не хватает памяти.
- `JamSpell`: https://github.com/bakwc/JamSpell. Русская модель работает только под Linux (в Windows — через WSL). Можно обучить свою модель, которая будет работать в Windows. Быстр, но качество сомнительное: например, исправил слово "томленый" на "атомный".
- `Yandex.Speller` через библиотеку `pyaspeller`: работает быстро, коверканий не замечено. Недостатки — подключение через `speller.yandex.net`, что требует стабильного соединения, и неспособность исправлять простые ошибки вроде "полха" → "полка".

---

**Что можно улучшить**

1. Общая проблема всех вышеуказанных корректоров — исправление по корпусу, где нет слов из узкоспециализированной области (например, связанных с корпорацией или миром Ведьмака).

    При наличии дополнительных ресурсов и времени лучшим вариантом было бы прогнать текст через Fine-tuned LLM, которая хорошо справляется с коррекцией ошибок и учитывает контекст. 

2. Извлечение именованных сущностей и их обработка. Можно реализовать с помощью `spacy`. К сожалению, вспомнил об этом только под конец.

---

In [3]:
from tqdm import tqdm
from preprocessing import (
    to_lower,
    tokenize,
    en_ru_mapping,
    replace_numbers,
    only_ru_chars,
    remove_repeating_letters,
    remove_stop_words,
    spell_correction,
    stemming,
    lemmatize,
    remove_freqs,
)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Loassar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Loassar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def process_sentences(sentences):
    processed_sentences = []
    pipeline = [
        tokenize,
        to_lower,
        en_ru_mapping,
        replace_numbers,
        only_ru_chars,
        remove_repeating_letters,
        # stemming,
        spell_correction,
        tokenize,
        lemmatize,
        remove_stop_words,
    ]
    for sentence in tqdm(sentences, desc="Sentence processing"):
        for step in pipeline:
            sentence = step(sentence)
        processed_sentences.append(sentence)

    return processed_sentences

In [5]:
sentences = [comment['quote'] for comment in dataset]
processed_sentences = process_sentences(sentences)

Sentence processing: 100%|██████████| 959/959 [02:21<00:00,  6.79it/s]


In [6]:
for s in processed_sentences[0:5]:
    print(s)

['среди', 'ночь', 'район', 'час', 'упасть', 'полха', 'вода', 'стекляна', 'тара', 'тобыть', 'посреди', 'лужа', 'стекло', 'всё', 'это', 'утро', 'кстати', 'это', 'первыя', 'падать', 'както', 'неправильно', 'расчитывать', 'должный', 'выставлять', 'определёный', 'картинка', 'пара', 'грохаться', 'полка', 'бутылка']
['програм', 'повышение', 'квалификаци', 'гильдия', 'цмф', 'хитрый', 'хитрый', 'весь', 'хотеть', 'повышаться', 'свой', 'предпочтение', 'направление', 'гильдия', 'гильдия', 'толкать', 'свой', 'нужно', 'сотрудник', 'белый', 'змея', 'например', 'програм', 'гибкий', 'сотрудник', 'выбирать', 'нужно']
['мур', 'новиград', 'город', 'контраст', 'цмф', 'змс', 'человек', 'превратиться', 'кот', 'всё', 'занятый', 'никто', 'хотеть', 'протянуть', 'лапка', 'товарищ', 'помочь', 'груз', 'всё', 'напряженить', 'спешка', 'мур', 'ранний', 'хороший', 'работать', 'втроём', 'порознь']
['тип', 'полгода', 'ждать', 'установка', 'фильтр', 'маслобаза', 'думать', 'бесконечный', 'огненный', 'огонь', 'затягивать',

In [7]:
processed_sentences_without_freqs = remove_freqs(processed_sentences)

In [8]:
processed_sentences_without_freqs[0:5]

['среди ночь район час упасть полха вода стекляна тара тобыть посреди лужа стекло утро кстати первыя падать неправильно расчитывать должный выставлять определёный картинка пара грохаться полка бутылка',
 'програм повышение квалификаци гильдия хитрый хитрый хотеть повышаться предпочтение направление гильдия гильдия толкать нужно сотрудник белый змея например програм гибкий сотрудник выбирать нужно',
 'новиград город контраст превратиться кот занятый никто хотеть протянуть лапка товарищ помочь груз напряженить спешка ранний втроём порознь',
 'полгода ждать установка фильтр маслобаза думать бесконечный огненный огонь затягивать поставка давно порядок',
 'ранний бумажка летать теряться искать приходиться зелёный место плантире информация рука ничо пропалаять стать гораздо удобный']

**ТЕМАТИЧЕСКОЕ МОДЕЛИРОВАНИЕ**
---

---

**Задачи:**
- Определить количество тематик, встречающихся в тексте, и дать им описание;
- Сопоставить каждый комментарий работника с набором тематик;
- Разработать модель, определяющую тематику ранее неизвестных комментариев.

**Решение:**
Необходимо решить задачу тематического моделирования. Наиболее популярные варианты:

1. `LDA`: 
    - Почему я решил не применять `LDA`?
    - Для `LDA` необходимо заранее определить количество тем, что вызывает сложности. Существует несколько способов определения оптимального количества кластеров (в данном случае — тематик), но ни разу не удалось получить результат, отличный от двойки. Как мне кажется, это связано с тем, что тривиальный случай — разделение комментариев на положительные и отрицательные.
    - Этот подход мог бы сработать при наличии эвристик. В данном случае такими эвристиками могли бы стать вопросы, задаваемые работникам, и идеи, заложенные в их основе (которые мог описать человек, их составлявший).

2. `BERTopic` https://maartengr.github.io/BERTopic/index.html:
    - Решение на основе трансформеров. Здесь идея проста (как палка и веревка): строим векторное представление (эмбеддинги) для каждого комментария, затем применяем метод кластеризации, после чего составляем описание для каждого кластера (название тематики).

    Двигаемся по порядку:
    1. Модель для построения эмбеддингов должна быть "легкой", быстрой, поддерживать русский язык (желательно с соответствующей специализацией) и создавать адекватные представления на малых данных (1–3 предложения). Я нашел лишь одну модель, которая популярна и соответствует всем моим требованиям — это `'cointegrated/rubert-tiny2'`.
    2. По умолчанию в `BERTopic` для кластеризации используется `HDBSCAN`. В целом эта модель хорошо подходит для задачи "слепой" кластеризации, когда неизвестно ни количество, ни примерный состав кластеров, поэтому я оставил ее, лишь немного поигравшись с параметрами.
    3. `UMAP` — это один из способов понижения размерности для эмбеддингов, используемый для оптимизации. Как правило, с ним у моделей кластеризации получается более качественный инференс, так как эти алгоритмы не слишком хорошо работают с данными высокой размерности.
    4. `CountVectorizer` отвечает непосредственно за репрезентативную часть кластеров. Вся часть, касающаяся обработки текста, была вынесена в разделе предобработки, поэтому здесь можно изменить разве что n-граммы.

---

In [9]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

sentence_model = SentenceTransformer('cointegrated/rubert-tiny2', device=device)

umap_model = UMAP(n_neighbors=20, n_components=4, min_dist=0.1, metric='cosine')

hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean', cluster_selection_method='leaf')

vectorizer_model = CountVectorizer(ngram_range=(1, 3))

topic_model = BERTopic(
    embedding_model=sentence_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    verbose=True,
    language="russian"
)

topics, probabilities = topic_model.fit_transform(processed_sentences_without_freqs)

c:\Users\Loassar\Documents\Career\GPN\final\gpn-ds-competition\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


2024-11-14 21:46:47,868 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 30/30 [00:00<00:00, 89.33it/s]
2024-11-14 21:46:48,217 - BERTopic - Embedding - Completed ✓
2024-11-14 21:46:48,217 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-14 21:46:53,500 - BERTopic - Dimensionality - Completed ✓
2024-11-14 21:46:53,500 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-14 21:46:53,510 - BERTopic - Cluster - Completed ✓
2024-11-14 21:46:53,520 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-14 21:46:53,604 - BERTopic - Representation - Completed ✓


In [10]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,449,-1_новый_стать_время_например,"[новый, стать, время, например, проблема, фени...",[цинтрийский феникс стараться обновление ценик...
1,0,69,0_скатя_снежный_зеркало_словно,"[скатя, снежный, зеркало, словно, дракон, испо...",[мантикорный трубка дело хорош перекачка эфект...
2,1,62,1_день_смена_ночной_делать,"[день, смена, ночной, делать, график, стоить, ...",[растроить девушка буквально декабрь проситься...
3,2,61,2_знать_образование_делать_опыт,"[знать, образование, делать, опыт, идея, безоп...",[думать малый знание начинать знать случай сре...
4,3,54,3_месяц_смена_момент_сделать,"[месяц, смена, момент, сделать, год, время, за...",[идея внутрений резерв сотрудник учёт перезда ...
5,4,41,4_новый_палантир_схема_приходиться,"[новый, палантир, схема, приходиться, предопла...",[новый схема оплата палантир честно кошмар кли...
6,5,40,5_зарплата_условие_карьерный_большой,"[зарплата, условие, карьерный, большой, рост, ...",[зарплата малый возможность рост ограничить по...
7,6,40,6_попасть_технолог_старший технолог_алхимик,"[попасть, технолог, старший технолог, алхимик,...",[высокий уровень программа стажировка молодой ...
8,7,34,7_ребёнок_квартал_премия_прошлый,"[ребёнок, квартал, премия, прошлый, год, пособ...",[прошлый квартал премия получить змз белый гор...
9,8,31,8_праздник_день рождение_рождение_день,"[праздник, день рождение, рождение, день, скат...",[ранний начать открывать простой колег друг др...


*Если сгенерировались только 2-3 тематики стоит перезапустить алгоритм*

**КЛАССИФИКАЦИЯ**
---

---

Теперь, когда датасет размечен, можно реализовать модель классификации комментариев по тематикам.

Из таблицы `get_topic_info` видно, что классы получились крайне несбалансированными: некоторые из них представлены выборками всего из 20 элементов, что затрудняет разбиение на тренировочную и тестовую выборки. 

Для решения этой проблемы использовал SMOTE (https://habr.com/ru/companies/otus/articles/782668/). Для построения эмбеддингов по-прежнему используется модель `'cointegrated/rubert-tiny2'`, которая работает хорошо, так зачем её менять? :)

Изначально планировалось использовать *бейзлайн* модель `RandomForestClassifier` для классификации, однако модель смогла показать высокие результаты как по accuracy (>95%), так и по остальным метрикам (что видно ниже), поэтому было решено остановиться на ней.

---

In [11]:
import pandas as pd

comments = [' '.join(s) for s in processed_sentences]

df = pd.DataFrame(topics, columns=['topic'])
df.insert(0, 'comments', comments)

print(df)

                                              comments  topic
0    среди ночь район час упасть полха вода стеклян...     -1
1    програм повышение квалификаци гильдия цмф хитр...      6
2    мур новиград город контраст цмф змс человек пр...      9
3    тип полгода ждать установка фильтр маслобаза д...     -1
4    всё просто ранний бумажка летать теряться иска...     -1
..                                                 ...    ...
954  город знать ночной смена работать это нормальн...     -1
955  праздник както както всё это очень организоват...      8
956        женский коллектив всё равно склока избежать      9
957  ещё нехват очень больший стоить наш помещение ...     -1
958  мур сначала вроде хершо палантирчик всё подсчи...     -1

[959 rows x 2 columns]


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoTokenizer, AutoModel
from imblearn.over_sampling import SMOTE

tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')
model = AutoModel.from_pretrained('cointegrated/rubert-tiny2')

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128, verbose=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

embeddings = df['comments'].apply(get_embedding)
X = list(embeddings)
y = df['topic']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9768835616438356
              precision    recall  f1-score   support

          -1       0.99      0.73      0.84        97
           0       0.92      0.99      0.96        97
           1       0.98      1.00      0.99        85
           2       0.97      1.00      0.98        91
           3       0.96      1.00      0.98        97
           4       1.00      1.00      1.00       101
           5       0.96      1.00      0.98        77
           6       0.98      1.00      0.99        95
           7       1.00      1.00      1.00        91
           8       0.97      1.00      0.99        72
           9       0.99      1.00      0.99        84
          10       1.00      1.00      1.00        82
          11       0.99      1.00      0.99        99

    accuracy                           0.98      1168
   macro avg       0.98      0.98      0.98      1168
weighted avg       0.98      0.98      0.98      1168



**ЗАКЛЮЧЕНИЕ**
---

---

В заключение скажу, что выбранный подход к решению поставленной задачи отвечает всем заданным техническим и бизнес-требованиям и, по мнению автора, считается оптимальным с учетом приведенных ниже возможностей для улучшений и оптимизаций.

Далее предложены варианты улучшения системы:

1. Первый этап — предобработка текстовых данных.

    Здесь я бы улучшил способ обработки грамматических ошибок в тексте, например, используя `Fine-tuned LLM`. Текст без ошибок лучше стандартизируется с помощью техник лемматизации, что, в свою очередь, улучшает качество работы моделей тематического моделирования. Также важно обрабатывать именованные сущности отдельно.

2. Второй этап — тематическое моделирование.

    Как я описывал ранее, качество работы алгоритмов тематического моделирования можно повысить за счет добавления эвристик, например списка потенциальных тематик. Также можно попробовать использовать более "тяжелые" модели для генерации эмбеддингов и протестировать большое количество параметров для подбора оптимальных значений.

3. Третий этап — классификация.

    Здесь есть значительный простор для улучшений, так как используется простая базовая модель. Как минимум, ее можно заменить на многослойный перцептрон, но я не уверен в необходимости этого.

---
